In [29]:
import os, json
import numpy as np
import pickle


In [67]:
l = os.listdir("/data/yingshac/MMMHQA/distractors")
max_idx = max([int(i.split('.')[0]) for i in l])
print("max_idx = ", max_idx)
x = np.full(max_idx+1, -1, dtype=np.longlong)
print(x.shape)
with open("/data/yingshac/maskrcnn-benchmark-1/datasets1/visualgenome/visualgenome/neg_hw.lineidx", "r") as fp:
    hw_lineidx = [int(i.strip()) for i in fp.readlines()]
with open("/data/yingshac/maskrcnn-benchmark-1/datasets1/visualgenome/visualgenome/neg_img.lineidx", "r") as fp:
    img_lineidx = [int(i.strip()) for i in fp.readlines()]
with open("/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_338842/predictions.lineidx", "r") as fp:
    pred_lineidx = [int(i.strip()) for i in fp.readlines()]
assert len(hw_lineidx) == len(img_lineidx) == len(pred_lineidx)

max_idx =  10340193
(10340194,)


In [71]:
# Generate lineidx with the correct order
count = 0
x = pickle.load(open("./Fix_VinVL_predictions_order_neg_imgs_tmp.pkl", "wb"))
x[]
with open("/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_338842/predictions.tsv", "r") as fp:
    for i in range(305000, len(pred_lineidx)):
        fp.seek(pred_lineidx[i])
        imgid = int(fp.readline().strip().split('\t')[0])
        x[imgid] = pred_lineidx[i]
        count += 1
        if count % 5000 == 0: print(count)

5000
10000
15000
20000
25000
30000


In [76]:
len(pred_lineidx)

338842

In [72]:
pickle.dump(x, open("./Fix_VinVL_predictions_order_neg_imgs_tmp.pkl", "wb"))

In [84]:
# Write to lineidx file.
# write -1 if the corresponding id doesn't have an associated img
count = 0
with open('/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_338842/predictions_cor.lineidx', "w") as fp:
    for i in x:
        fp.write('{0}'.format(i) + '\n')
        if i == -1: count += 1
print("num of non-existing imgs = ", count)

num of non-existing imgs =  0


In [49]:
# write a new predictions.tsv according to the lineidx created
with open("/data/yingshac/MMMHQA/VinVL_output/gold_0_22265/predictions.tsv", "r") as fp, open("/data/yingshac/MMMHQA/VinVL_output/gold_0_22265/predictions_cor.tsv", "w") as fp_cor:
    for i in range(len(x)):
        fp.seek(x[i])
        fp_cor.write(fp.readline())
        if i % 1000 == 999: print(i)

999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999


In [50]:
# generate new lineidx based on predictions_cor
### given a file, store lineidx in <filename>.lineidx in the same directory
directory = '/data/yingshac/MMMHQA/VinVL_output/gold_0_22265/'
filename = 'predictions_cor.tsv'

fp = open(os.path.join(directory, filename), 'r')
linefp = open(os.path.join(directory, '{}.lineidx'.format(filename.split('.')[0])), 'w')
count = 0
while True:
    if count %2000 == 1999: print(count)
    tmp = fp.tell()
    l = fp.readline()
    if not l: break
    linefp.write('{0}'.format(tmp) + '\n')
    count += 1
fp.close()
linefp.close()
print(count)

1999
3999
5999
7999
9999
11999
13999
15999
17999
19999
21999
22265


In [51]:
# Sanity check: there should be NO more mismatch
with open("/data/yingshac/MMMHQA/VinVL_output/gold_0_22265/predictions_cor.lineidx", "r") as fp:
    pred_lineidx_cor = [int(i.strip()) for i in fp.readlines()]
count = 0
with open("/data/yingshac/MMMHQA/VinVL_output/gold_0_22265/predictions_cor.tsv", "r") as fp:
    for i in pred_lineidx_cor:
        fp.seek(i)
        imgid = int(fp.readline().strip().split('\t')[0])
        if not imgid == count: print(count, imgid)
        count += 1
        if count % 2000 == 0: print(count)

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000


In [52]:
# Concat two tsv files, adjust lineidx correspondingly
tsv_list = ["/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_108000/inference/vinvl_vg_x152c4/predictions.tsv",
            "/data/yingshac/MMMHQA/VinVL_output/neg_imgs_108000/inference/vinvl_vg_x152c4/predictions.tsv"]
lineidx_list = ["/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_108000/inference/vinvl_vg_x152c4/predictions.lineidx",
            "/data/yingshac/MMMHQA/VinVL_output/neg_imgs_108000/inference/vinvl_vg_x152c4/predictions.lineidx"]
output_path = "/data/yingshac/MMMHQA/VinVL_output/neg_imgs_0_338842"
if not os.path.isdir(output_path): os.mkdir(output_path)

offset = 0
with open(os.path.join(output_path, "predictions.tsv"), "w") as output_tsv, open(os.path.join(output_path, "predictions.lineidx"), "w") as output_lineidx:
    for T, L in zip(tsv_list, lineidx_list):
        with open(L, "r") as fp:
            lineidx = [int(i.strip()) for i in fp.readlines()]
        print("len(lineidx) = ", len(lineidx))
        with open(T, "r") as fp:
            for i in range(len(lineidx)):
                fp.seek(lineidx[i])
                output_tsv.write(fp.readline())
                output_lineidx.write('{0}'.format(i+offset) + '\n')
                if i%5000 == 4999: print(i)
        offset += output_tsv.tell()
        print(offset, i)
        

15192064999
16017534999
33138744999
34958739999
46396489999
46816934999
60023204999
62642639999
90883149999
90896879999
103138104999
112610054999
130003254999
172005679999
180697544999
183300214999
199152774999
207182684999
209369807640 209367846579
2339169999
26017469999
27784874999
28881754999
68175694999
69194129999
85226489999
88111699999
91249654999
94489969999
98605634999
109248449999
119529499999
125771594999
131166029999
135073474999
151189829999
173231899999
182324599999
182715524999
186505359999
198782044999
201299599999
211888889999
219501229999
219874424999
236713899999
239791949999
259524804999
263288824999
264389789999
272322304999
286359834999
287389259999
290421769999
308459164999
316943849999
336747499999
343209499999
352245259999
371784724999
388215684999
396600339999
399686729999
401903769999
403834234999
418999969999
431023069999
443246244999
443540099999
865616043077 446874466626
